# import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import os

seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

/app/user1/miniconda3/envs/pytorchv1/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# data download and load
https://drive.google.com/drive/folders/1ZTsiah25xqdNVz9kxE4-tHAG2uSbF-AC?usp=drive_link

In [2]:
train_inputs = np.load("AI/Taichi data/train_inputs.npy")
train_labels = np.load("AI/Taichi data/train_labels.npy")
val_inputs = np.load("AI/Taichi data/val_inputs.npy")
val_labels = np.load("AI/Taichi data/val_labels.npy")

In [3]:
class Feeder(torch.utils.data.Dataset):
  def __init__(self, data_path, label_path):
      super().__init__()
      self.label = np.load(label_path)
      self.data = np.load(data_path)

  def __len__(self):
      return len(self.label)

  def __iter__(self):
      return self

  def __getitem__(self, index):
      data = np.array(self.data[index])
      label = self.label[index]
      return data, label

# graph structure

In [4]:
class Graph():
    def __init__(self, hop_size):
        self.get_edge()
        self.hop_size = hop_size
        self.hop_dis = self.get_hop_distance(self.num_node, self.edge, hop_size=hop_size)
        self.get_adjacency()

    def __str__(self):
        return self.A

    def get_edge(self):
        self.num_node = 17
        self_link = [(i, i) for i in range(self.num_node)]  # Loop
        neighbor_link = [(0, 7), (7, 8), (8, 9), (9, 10),
                         (8, 11), (11, 12), (12, 13),
                         (8, 14), (14, 15), (15, 16),
                         (0, 1), (1, 2), (2, 3),
                         (0, 4), (4, 5), (5, 6)]

        self.edge = self_link + neighbor_link

    def get_adjacency(self):
        valid_hop = range(0, self.hop_size + 1, 1)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = self.normalize_digraph(adjacency)
        A = np.zeros((len(valid_hop), self.num_node, self.num_node))
        for i, hop in enumerate(valid_hop):
            A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis == hop]
        self.A = A

    def get_hop_distance(self, num_node, edge, hop_size):
        A = np.zeros((num_node, num_node))
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1
        hop_dis = np.zeros((num_node, num_node)) + np.inf
        transfer_mat = [np.linalg.matrix_power(A, d) for d in range(hop_size + 1)]
        arrive_mat = (np.stack(transfer_mat) > 0)
        for d in range(hop_size, -1, -1):
            hop_dis[arrive_mat[d]] = d
        return hop_dis

    def normalize_digraph(self, A):
        Dl = np.sum(A, 0)
        num_node = A.shape[0]
        Dn = np.zeros((num_node, num_node))
        for i in range(num_node):
            if Dl[i] > 0:
                Dn[i, i] = Dl[i] ** (-1)
        DAD = np.dot(A, Dn)
        return DAD


class Sparse_Graph():
    def __init__(self, hop_size):
        self.get_edge()
        self.hop_size = hop_size
        self.hop_dis = self.get_hop_distance(self.num_node, self.edge, hop_size=hop_size)
        self.get_adjacency()

    def __str__(self):
        return str(self.A)

    def get_edge(self):
        self.num_node = 25
        self_link = [(i, i) for i in range(self.num_node)] # Loop
        neighbor_base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                          (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                          (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                          (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                          (22, 23), (23, 8), (24, 25), (25, 12)]
        neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_base]
        self.edge = self_link + neighbor_link

    def get_adjacency(self):
        valid_hop = range(0, self.hop_size + 1, 1)
        adjacency = sp.lil_matrix((self.num_node, self.num_node)) # 稀疏邻接矩阵
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = self.normalize_digraph(adjacency)
        A = np.zeros((len(valid_hop), self.num_node, self.num_node))
        for i, hop in enumerate(valid_hop):
            A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis == hop]
        self.A = A

    def get_hop_distance(self, num_node, edge, hop_size):
        A = sp.lil_matrix((num_node, num_node))
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1
        hop_dis = np.zeros((num_node, num_node)) + np.inf
        transfer_mat = [np.linalg.matrix_power(A.toarray(), d) for d in range(hop_size + 1)]
        arrive_mat = (np.stack(transfer_mat) > 0)
        for d in range(hop_size, -1, -1):
            hop_dis[arrive_mat[d]] = d
        return hop_dis

    def normalize_digraph(self, A):
        Dl = np.sum(A, 0)
        num_node = A.shape[0]
        Dn = np.zeros((num_node, num_node))
        for i in range(num_node):
            if Dl[i] > 0:
                Dn[i, i] = Dl[i]**(-1)
        DAD = np.dot(A.toarray(), Dn)
        return DAD

# SGC, SSGC, SGCPP, TGCPP

In [5]:
class SpatialGraphConvolution(nn.Module):
  def __init__(self, in_channels, out_channels, s_kernel_size):
    super().__init__()
    self.s_kernel_size = s_kernel_size
    self.conv = nn.Conv2d(in_channels=in_channels,
                          out_channels=out_channels * s_kernel_size,
                          kernel_size=1)

  def forward(self, x, A):
    x = self.conv(x)
    n, kc, t, v = x.size()
    x = x.view(n, self.s_kernel_size, kc//self.s_kernel_size, t, v)
    x = torch.einsum('nkctv,kvw->nctw', (x, A))
    return x.contiguous()

class SpatialSeparateGraphConvolution(nn.Module):
  def __init__(self, in_channels, out_channels, s_kernel_size):
    super().__init__()
    self.s_kernel_size = s_kernel_size
    self.deep_wise_conv = nn.Conv2d(in_channels=in_channels,
                out_channels=in_channels,
                kernel_size=3,
                padding=1,
                groups=in_channels,
                )
    self.point_wise_conv = nn.Conv2d(in_channels=in_channels,
                    out_channels=out_channels * s_kernel_size, kernel_size=1)


  def forward(self, x, A):
    x = self.deep_wise_conv(x)
    x = self.point_wise_conv(x)
    n, kc, t, v = x.size()
    x = x.view(n, self.s_kernel_size, kc//self.s_kernel_size, t, v)
    x = torch.einsum('nkctv,kvw->nctw', (x, A))
    return x.contiguous()

SGCPP, TGCPP in [ST-GCN++](https://arxiv.org/pdf/2205.09443)

In [6]:
class SpatialGraphConvolutionPlusPlus(nn.Module):
  def __init__(self, in_channels, out_channels, s_kernel_size):
    super().__init__()
    self.s_kernel_size = s_kernel_size
    self.conv = nn.Conv2d(in_channels=in_channels,
                          out_channels=out_channels * s_kernel_size,
                          kernel_size=1)

  def forward(self, x, A):
    # print("SGC_0: ", x.shape)
    y = self.conv(x)
    n, kc, t, v = y.size()
    y = y.view(n, self.s_kernel_size, kc//self.s_kernel_size, t, v)
    y = torch.einsum('nkctv,kvw->nctw', (y, A))
    if y.shape == x.shape:
        y += x
    # print("SGC_1: ", x.shape)
    # print("SGC_2: ", x.contiguous().shape)
    return y.contiguous()

class TemporalGraphConvolutionPlusPlus(nn.Module):
    def __init__(self, out_channels):
        super().__init__()
        self.out_channels = out_channels
        self.cv1_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv2_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv3_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv4_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv5_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv6_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)

        self.cv2_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=1, padding=(1,0))
        self.cv3_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=2, padding=(2,0))
        self.cv4_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=3, padding=(3,0))
        self.cv5_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=4, padding=(4,0))
        self.cv6_2 = nn.MaxPool2d(kernel_size=(3,1),stride=(1,1),padding=(1,0))

        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x1 = self.cv1_1(x)
        x2 = self.cv2_2(self.cv2_1(x))
        x3 = self.cv3_2(self.cv3_1(x))
        x4 = self.cv4_2(self.cv4_1(x))
        x5 = self.cv5_2(self.cv5_1(x))
        x6 = self.cv6_2(self.cv6_1(x))
        y = x1+x2+x3+x4+x5+x6
        y = self.relu(self.bn(y))
        return y

SSGC+SGCPP=SSGCPP, TDGC+TGCPP=TDGCPP

In [7]:
class SpatialSeparateGraphConvolutionPlusPlus(nn.Module):
  def __init__(self, in_channels, out_channels, s_kernel_size):
    super().__init__()
    self.s_kernel_size = s_kernel_size
    self.deep_wise_conv = nn.Conv2d(in_channels=in_channels,
                out_channels=in_channels,
                kernel_size=3,
                padding=1,
                groups=in_channels,
                )
    self.point_wise_conv = nn.Conv2d(in_channels=in_channels,
                    out_channels=out_channels * s_kernel_size, kernel_size=1)


  def forward(self, x, A):
    y = self.deep_wise_conv(x)
    y = self.point_wise_conv(y)
    n, kc, t, v = y.size()
    y = y.view(n, self.s_kernel_size, kc//self.s_kernel_size, t, v)
    y = torch.einsum('nkctv,kvw->nctw', (y, A))
    if y.shape == x.shape:
        y+=x
    return y.contiguous()

class TemporalDilationGraphConvolutionPlusPlus(nn.Module):
    def __init__(self, out_channels):
        super().__init__()
        self.out_channels = out_channels
        self.cv1_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv2_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv3_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv4_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv5_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)
        self.cv6_1 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(1,1), stride=1)

        self.cv2_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=1, padding=(1,0))
        self.cv3_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=2, padding=(2,0))
        self.cv4_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=3, padding=(3,0))
        self.cv5_2 = nn.Conv2d(in_channels = out_channels, out_channels = out_channels, kernel_size=(3,1), stride=1, dilation=4, padding=(4,0))
        self.cv6_2 = nn.MaxPool2d(kernel_size=(3,1),stride=(1,1),padding=(1,0))

        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x1 = self.cv1_1(x)
        x2 = self.cv2_2(self.cv2_1(x))
        x3 = self.cv3_2(self.cv3_1(x))
        x4 = self.cv4_2(self.cv4_1(x))
        x5 = self.cv5_2(self.cv5_1(x))
        x6 = self.cv6_2(self.cv6_1(x))
        y = x1+x2+x3+x4+x5+x6
        y = self.relu(self.bn(y))
        return y

# STGC, SSTGC, STDGC, SSTDGC, STGCL, STGCPP, SSTDGCPP blocks

In [8]:
class STGC_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    self.sgc = SpatialGraphConvolution(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    self.M = nn.Parameter(torch.ones(A_size))

    self.tgc = nn.Sequential(nn.BatchNorm2d(out_channels),
                            nn.ReLU(),
                            nn.Dropout(dropout),
                            nn.Conv2d(out_channels,
                                      out_channels,
                                      (t_kernel_size, 1), # kernel_size
                                      (stride, 1), # stride
                                      ((t_kernel_size - 1) // 2, 0), # padding
                                      ),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU())

  def forward(self, x, A):
    x = self.tgc(self.sgc(x, A * self.M))
    return x

class SSTGC_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    self.sgc = SpatialSeparateGraphConvolution(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    self.M = nn.Parameter(torch.ones(A_size))

    self.tgc = nn.Sequential(nn.BatchNorm2d(out_channels),
                            nn.ReLU(),
                            nn.Dropout(dropout),
                            nn.Conv2d(out_channels,
                                      out_channels,
                                      (t_kernel_size, 1), # kernel_size
                                      (stride, 1), # stride
                                      ((t_kernel_size - 1) // 2, 0), # padding
                                      ),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU())

  def forward(self, x, A):
    x = self.tgc(self.sgc(x, A * self.M))
    return x

class STDGC_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    self.sgc = SpatialGraphConvolution(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    # Learnable weight matrix M 给边缘赋予权重。学习哪个边是重要的。
    self.M = nn.Parameter(torch.ones(A_size))

    self.tgc = nn.Sequential(nn.BatchNorm2d(out_channels),
                            nn.ReLU(),
                            nn.Dropout(dropout),
                            nn.Conv2d(out_channels,
                                      out_channels,
                                      (t_kernel_size, 1), # kernel_size
                                      (stride, 1), # stride
                                      ((t_kernel_size - 1) // 2, 0), # padding
                                      dilation = 2,
                                      ),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU())

  def forward(self, x, A):
    x = self.tgc(self.sgc(x, A * self.M))
    return x

class SSTDGC_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    self.sgc = SpatialSeparateGraphConvolution(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    # Learnable weight matrix
    self.M = nn.Parameter(torch.ones(A_size))

    self.tgc = nn.Sequential(nn.BatchNorm2d(out_channels),
                            nn.ReLU(),
                            nn.Dropout(dropout),
                            nn.Conv2d(out_channels,
                                      out_channels,
                                      (t_kernel_size, 1), # kernel_size
                                      (stride, 1), # stride
                                      ((t_kernel_size - 1) // 2, 0), # padding
                                      dilation = 2,
                                      ),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU())

  def forward(self, x, A):
    x = self.tgc(self.sgc(x, A * self.M))
    return x

STGL_block

In [9]:
class STGCL_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    self.sgc = SpatialGraphConvolution(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    self.M = nn.Parameter(torch.ones(A_size)) # 论文中的A
    self.K = nn.Parameter(torch.ones(A_size)) # 论文中的K

    self.tgc = nn.Sequential(nn.BatchNorm2d(out_channels),
                            nn.ReLU(),
                            nn.Dropout(dropout),
                            nn.Conv2d(out_channels,
                                      out_channels,
                                      (t_kernel_size, 1), # kernel_size
                                      (stride, 1), # stride
                                      ((t_kernel_size - 1) // 2, 0), # padding
                                      ),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU())

  def forward(self, x, A):
    x = self.tgc(self.sgc(x, torch.mul(A,self.M) + self.K))
    return x

STGC++ block

In [10]:
class STGCPP_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    self.sgcpp = SpatialGraphConvolutionPlusPlus(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    self.M = nn.Parameter(torch.ones(A_size))

    """
    Cite from https://arxiv.org/pdf/2205.09443 :
    "We use a multi-branch temporal ConvNet (TCN) to replace the single branch design.
    The adopted multi-branch TCN consists of six branches: a ‘1x1’ Conv branch, a Max-Pooling branch,
    and four temporal 1D Conv branches with kernel size 3 and dilations from 1 to 4. It first transforms features
    with ‘1x1’ Conv and divides them into six groups with equal channel width.
    Then, each feature group is processed with a single branch. The six outputs are concatenated together
    and processed by another ‘1x1’ Conv to form the output of the multi-branch TCN.
    The new TCN design not only improves the temporal modeling capabililty,
    but also saves the computational cost and parameters, due to the reduced channel width for every single branch."
    """
    self.tgcpp =  TemporalGraphConvolutionPlusPlus(out_channels)

  def forward(self, x, A):
    x = self.tgcpp(self.sgcpp(x, A * self.M))
    return x

SSTDGC++ block

In [11]:
class SSTDGCPP_block(nn.Module):
  def __init__(self, in_channels, out_channels, stride, t_kernel_size, A_size, dropout=0.5):
    super().__init__()
    self.ssgcpp = SpatialSeparateGraphConvolutionPlusPlus(in_channels=in_channels,
                                       out_channels=out_channels,
                                       s_kernel_size=A_size[0])

    self.M = nn.Parameter(torch.ones(A_size))
    self.tdgcpp = TemporalDilationGraphConvolutionPlusPlus(out_channels)

  def forward(self, x, A):
    x = self.tdgcpp(self.ssgcpp(x, A * self.M))
    return x

# models

ST_GCN, SST_GCN, STD_GCN, SSTD_GCN (ours model)

In [12]:
class ST_GCN(nn.Module):
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graph制作
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1]) # 75

    # STGC_blocks
    self.stgc1 = STGC_block(in_channels, 32, 1, t_kernel_size, A_size) # in_c=3, t_k_s= 9， 1是步长
    self.stgc2 = STGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc3 = STGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc4 = STGC_block(32, 64, 2, t_kernel_size, A_size)
    self.stgc5 = STGC_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc6 = STGC_block(64, 64, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(64, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node

    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)

    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x


class SST_GCN(nn.Module):
  # Spacial Separate Temporal GCN
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graph制作
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1]) # 75

    # STGC_blocks
    self.stgc1 = SSTGC_block(in_channels, 32, 1, t_kernel_size, A_size) # in_c=3, t_k_s= 9， 1是步长
    self.stgc2 = SSTGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc3 = SSTGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc4 = SSTGC_block(32, 64, 2, t_kernel_size, A_size)
    self.stgc5 = SSTGC_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc6 = SSTGC_block(64, 64, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(64, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node

    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)

    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x


class STD_GCN(nn.Module):
  # Spacial Temporal Dilation GCN
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graph制作
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1]) # 75

    # STGC_blocks
    self.stgc1 = STDGC_block(in_channels, 32, 1, t_kernel_size, A_size) # in_c=3, t_k_s= 9， 1是步长
    self.stgc2 = STDGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc3 = STDGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc4 = STDGC_block(32, 64, 2, t_kernel_size, A_size)
    self.stgc5 = STDGC_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc6 = STDGC_block(64, 64, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(64, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node

    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)

    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x


class SSTD_GCN(nn.Module):
  # Spacial Temporal Dilation GCN
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graph制作
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1]) # 75

    # STGC_blocks
    self.stgc1 = SSTDGC_block(in_channels, 32, 1, t_kernel_size, A_size) # in_c=3, t_k_s= 9， 1是步长
    self.stgc2 = SSTDGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc3 = SSTDGC_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc4 = SSTDGC_block(32, 64, 2, t_kernel_size, A_size)
    self.stgc5 = SSTDGC_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc6 = SSTDGC_block(64, 64, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(64, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node

    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)

    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x

STGL_GCN(others, 2023)

In [13]:
# https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=10047922
class STGL_GCN(nn.Module):
  # Spacial Temporal Dilation GCN
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graph制作
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    # D = torch.tensor(torch.randn_like(graph.A), dtype=torch.float32, requires_grad=False)
    # E = torch.tensor(torch.randn_like(graph.A), dtype=torch.float32, requires_grad=False)

    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1]) # 75

    # STGC_blocks
    self.stgc1 = STGCL_block(in_channels, 64, 1, t_kernel_size, A_size) # in_c=3, t_k_s= 9， 1是步长
    self.stgc2 = STGCL_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc3 = STGCL_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc4 = STGCL_block(64, 128, 2, t_kernel_size, A_size)
    self.stgc5 = STGCL_block(128, 128, 1, t_kernel_size, A_size)
    self.stgc6 = STGCL_block(128, 128, 1, t_kernel_size, A_size)
    self.stgc7 = STGCL_block(128, 256, 2, t_kernel_size, A_size)
    self.stgc8 = STGCL_block(256, 256, 1, t_kernel_size, A_size)
    self.stgc9 = STGCL_block(256, 256, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(256, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node

    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)
    x = self.stgc7(x, self.A)
    x = self.stgc8(x, self.A)
    x = self.stgc9(x, self.A)


    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x

STGCN++(others, 2022)

In [14]:
# https://arxiv.org/pdf/2205.09443
class ST_GCNPP(nn.Module):
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graph制作
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1]) # 75

    # STGC_blocks
    # 需要把STGC模块中的sgc添加残差链接，tgc添加
    self.stgc1 = STGCPP_block(in_channels, 32, 1, t_kernel_size, A_size) # in_c=3, t_k_s= 9， 1是步长
    self.stgc2 = STGCPP_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc3 = STGCPP_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc4 = STGCPP_block(32, 64, 2, t_kernel_size, A_size)
    self.stgc5 = STGCPP_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc6 = STGCPP_block(64, 64, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(64, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node

    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)

    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x

SSTD-GCN++: STGCN++ with SSTDGCN (Ours is embedded in theirs)

In [15]:
class SSTD_GCNPP(nn.Module):
  def __init__(self, num_classes, in_channels, t_kernel_size, hop_size):
    super().__init__()
    # graph制作
    graph = Graph(hop_size)
    A = torch.tensor(graph.A, dtype=torch.float32, requires_grad=False)
    self.register_buffer('A', A)
    A_size = A.size()

    # Batch Normalization
    self.bn = nn.BatchNorm1d(in_channels * A_size[1]) # 75

    # STGC_blocks
    # 需要把STGC模块中的sgc添加残差链接，tgc添加
    self.stgc1 = SSTDGCPP_block(in_channels, 32, 1, t_kernel_size, A_size) # in_c=3, t_k_s= 9， 1是步长
    self.stgc2 = SSTDGCPP_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc3 = SSTDGCPP_block(32, 32, 1, t_kernel_size, A_size)
    self.stgc4 = SSTDGCPP_block(32, 64, 2, t_kernel_size, A_size)
    self.stgc5 = SSTDGCPP_block(64, 64, 1, t_kernel_size, A_size)
    self.stgc6 = SSTDGCPP_block(64, 64, 1, t_kernel_size, A_size)

    # Prediction
    self.fc = nn.Conv2d(64, num_classes, kernel_size=1)

  def forward(self, x):
    # Batch Normalization
    N, C, T, V = x.size() # batch, channel, frame, node

    x = x.permute(0, 3, 1, 2).contiguous().view(N, V * C, T)
    x = self.bn(x)
    x = x.view(N, V, C, T).permute(0, 2, 3, 1).contiguous()

    # STGC_blocks
    x = self.stgc1(x, self.A)
    x = self.stgc2(x, self.A)
    x = self.stgc3(x, self.A)
    x = self.stgc4(x, self.A)
    x = self.stgc5(x, self.A)
    x = self.stgc6(x, self.A)

    # Prediction
    x = F.avg_pool2d(x, x.size()[2:])
    x = x.view(N, -1, 1, 1)
    x = self.fc(x)
    x = x.view(x.size(0), -1)
    return x

# Train

## ST-GCN(Top1 Acc=71.2500)

In [22]:
# ST-GCN
NUM_EPOCH = 200
BATCH_SIZE = 256
acc_list = []
val_acc_list = []
best_acc = 0
val_best_acc = 0
model = ST_GCN(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.0140 | Accuracy: 4.1439 | best acc: 4.1439
# val : Epoch: 1 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 2 | Loss: 0.0139 | Accuracy: 4.2530 | best acc: 4.2530
# val : Epoch: 2 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 3 | Loss: 0.0138 | Accuracy: 7.6336 | best acc: 7.6336
# val : Epoch: 3 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 4 | Loss: 0.0136 | Accuracy: 8.6150 | best acc: 8.6150
# val : Epoch: 4 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 5 | Loss: 0.0134 | Accuracy: 8.2879 | best acc: 8.6150
# val : Epoch: 5 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 6 | Loss: 0.0133 | Accuracy: 9.0513 | best acc: 9.0513
# val : Epoch: 6 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 7 | Loss: 0.0132 | Accuracy: 8.0698 | best acc: 9.0513
# val : Epoch: 7 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 8 | Loss: 0.0131 | Accuracy: 8.0698 | best acc: 9.0513
# val : Epoch: 8 | Accuracy: 4.1667 | best acc:

# Train: Epoch: 64 | Loss: 0.0077 | Accuracy: 47.2192 | best acc: 47.2192
# val : Epoch: 64 | Accuracy: 10.8333 | best acc: 25.0000
# Train: Epoch: 65 | Loss: 0.0076 | Accuracy: 45.6925 | best acc: 47.2192
# val : Epoch: 65 | Accuracy: 12.5000 | best acc: 25.0000
# Train: Epoch: 66 | Loss: 0.0075 | Accuracy: 47.7644 | best acc: 47.7644
# val : Epoch: 66 | Accuracy: 14.5833 | best acc: 25.0000
# Train: Epoch: 67 | Loss: 0.0074 | Accuracy: 50.3817 | best acc: 50.3817
# val : Epoch: 67 | Accuracy: 17.0833 | best acc: 25.0000
# Train: Epoch: 68 | Loss: 0.0073 | Accuracy: 50.0545 | best acc: 50.3817
# val : Epoch: 68 | Accuracy: 17.5000 | best acc: 25.0000
# Train: Epoch: 69 | Loss: 0.0072 | Accuracy: 50.0545 | best acc: 50.3817
# val : Epoch: 69 | Accuracy: 23.3333 | best acc: 25.0000
# Train: Epoch: 70 | Loss: 0.0072 | Accuracy: 50.0545 | best acc: 50.3817
# val : Epoch: 70 | Accuracy: 25.8333 | best acc: 25.8333
# Train: Epoch: 71 | Loss: 0.0070 | Accuracy: 51.4722 | best acc: 51.4722
# 

# Train: Epoch: 126 | Loss: 0.0034 | Accuracy: 81.2432 | best acc: 83.5333
# val : Epoch: 126 | Accuracy: 30.4167 | best acc: 34.1667
# Train: Epoch: 127 | Loss: 0.0032 | Accuracy: 83.0971 | best acc: 83.5333
# val : Epoch: 127 | Accuracy: 33.7500 | best acc: 34.1667
# Train: Epoch: 128 | Loss: 0.0031 | Accuracy: 84.4057 | best acc: 84.4057
# val : Epoch: 128 | Accuracy: 30.8333 | best acc: 34.1667
# Train: Epoch: 129 | Loss: 0.0032 | Accuracy: 85.2781 | best acc: 85.2781
# val : Epoch: 129 | Accuracy: 31.6667 | best acc: 34.1667
# Train: Epoch: 130 | Loss: 0.0031 | Accuracy: 84.0785 | best acc: 85.2781
# val : Epoch: 130 | Accuracy: 33.3333 | best acc: 34.1667
# Train: Epoch: 131 | Loss: 0.0031 | Accuracy: 84.7328 | best acc: 85.2781
# val : Epoch: 131 | Accuracy: 41.6667 | best acc: 41.6667
# Train: Epoch: 132 | Loss: 0.0031 | Accuracy: 83.6423 | best acc: 85.2781
# val : Epoch: 132 | Accuracy: 30.0000 | best acc: 41.6667
# Train: Epoch: 133 | Loss: 0.0030 | Accuracy: 84.2966 | best 

# Train: Epoch: 188 | Loss: 0.0013 | Accuracy: 94.3293 | best acc: 96.7285
# val : Epoch: 188 | Accuracy: 50.8333 | best acc: 62.0833
# Train: Epoch: 189 | Loss: 0.0014 | Accuracy: 92.4755 | best acc: 96.7285
# val : Epoch: 189 | Accuracy: 55.4167 | best acc: 62.0833
# Train: Epoch: 190 | Loss: 0.0013 | Accuracy: 94.0022 | best acc: 96.7285
# val : Epoch: 190 | Accuracy: 53.3333 | best acc: 62.0833
# Train: Epoch: 191 | Loss: 0.0012 | Accuracy: 94.6565 | best acc: 96.7285
# val : Epoch: 191 | Accuracy: 46.2500 | best acc: 62.0833
# Train: Epoch: 192 | Loss: 0.0012 | Accuracy: 93.7841 | best acc: 96.7285
# val : Epoch: 192 | Accuracy: 56.6667 | best acc: 62.0833
# Train: Epoch: 193 | Loss: 0.0012 | Accuracy: 94.8746 | best acc: 96.7285
# val : Epoch: 193 | Accuracy: 59.5833 | best acc: 62.0833
# Train: Epoch: 194 | Loss: 0.0012 | Accuracy: 94.5474 | best acc: 96.7285
# val : Epoch: 194 | Accuracy: 56.6667 | best acc: 62.0833
# Train: Epoch: 195 | Loss: 0.0011 | Accuracy: 97.0556 | best 

## SST-GCN

In [26]:
# SST-GCN
NUM_EPOCH = 200
BATCH_SIZE = 256
acc_list1 = []
val_acc_list1 = []
best_acc = 0
val_best_acc = 0
model = SST_GCN(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list1.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list1.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.0140 | Accuracy: 4.3621 | best acc: 4.3621
# val : Epoch: 1 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 2 | Loss: 0.0139 | Accuracy: 4.9073 | best acc: 4.9073
# val : Epoch: 2 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 3 | Loss: 0.0137 | Accuracy: 7.0883 | best acc: 7.0883
# val : Epoch: 3 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 4 | Loss: 0.0135 | Accuracy: 7.6336 | best acc: 7.6336
# val : Epoch: 4 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 5 | Loss: 0.0134 | Accuracy: 7.5245 | best acc: 7.6336
# val : Epoch: 5 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 6 | Loss: 0.0133 | Accuracy: 8.3969 | best acc: 8.3969
# val : Epoch: 6 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 7 | Loss: 0.0132 | Accuracy: 8.9422 | best acc: 8.9422
# val : Epoch: 7 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 8 | Loss: 0.0131 | Accuracy: 9.7056 | best acc: 9.7056
# val : Epoch: 8 | Accuracy: 4.1667 | best acc:

# Train: Epoch: 64 | Loss: 0.0072 | Accuracy: 55.0709 | best acc: 55.0709
# val : Epoch: 64 | Accuracy: 29.5833 | best acc: 47.5000
# Train: Epoch: 65 | Loss: 0.0071 | Accuracy: 58.3424 | best acc: 58.3424
# val : Epoch: 65 | Accuracy: 41.6667 | best acc: 47.5000
# Train: Epoch: 66 | Loss: 0.0069 | Accuracy: 57.9062 | best acc: 58.3424
# val : Epoch: 66 | Accuracy: 39.1667 | best acc: 47.5000
# Train: Epoch: 67 | Loss: 0.0067 | Accuracy: 59.9782 | best acc: 59.9782
# val : Epoch: 67 | Accuracy: 40.0000 | best acc: 47.5000
# Train: Epoch: 68 | Loss: 0.0067 | Accuracy: 62.0502 | best acc: 62.0502
# val : Epoch: 68 | Accuracy: 45.4167 | best acc: 47.5000
# Train: Epoch: 69 | Loss: 0.0065 | Accuracy: 62.0502 | best acc: 62.0502
# val : Epoch: 69 | Accuracy: 44.1667 | best acc: 47.5000
# Train: Epoch: 70 | Loss: 0.0064 | Accuracy: 63.2497 | best acc: 63.2497
# val : Epoch: 70 | Accuracy: 42.9167 | best acc: 47.5000
# Train: Epoch: 71 | Loss: 0.0063 | Accuracy: 63.2497 | best acc: 63.2497
# 

# Train: Epoch: 126 | Loss: 0.0022 | Accuracy: 91.8212 | best acc: 92.9117
# val : Epoch: 126 | Accuracy: 81.6667 | best acc: 82.5000
# Train: Epoch: 127 | Loss: 0.0022 | Accuracy: 90.5125 | best acc: 92.9117
# val : Epoch: 127 | Accuracy: 84.1667 | best acc: 84.1667
# Train: Epoch: 128 | Loss: 0.0022 | Accuracy: 90.6216 | best acc: 92.9117
# val : Epoch: 128 | Accuracy: 77.0833 | best acc: 84.1667
# Train: Epoch: 129 | Loss: 0.0022 | Accuracy: 91.1668 | best acc: 92.9117
# val : Epoch: 129 | Accuracy: 75.0000 | best acc: 84.1667
# Train: Epoch: 130 | Loss: 0.0021 | Accuracy: 92.2574 | best acc: 92.9117
# val : Epoch: 130 | Accuracy: 79.5833 | best acc: 84.1667
# Train: Epoch: 131 | Loss: 0.0021 | Accuracy: 92.3664 | best acc: 92.9117
# val : Epoch: 131 | Accuracy: 83.3333 | best acc: 84.1667
# Train: Epoch: 132 | Loss: 0.0020 | Accuracy: 93.3479 | best acc: 93.3479
# val : Epoch: 132 | Accuracy: 82.9167 | best acc: 84.1667
# Train: Epoch: 133 | Loss: 0.0019 | Accuracy: 92.5845 | best 

# Train: Epoch: 188 | Loss: 0.0008 | Accuracy: 98.1461 | best acc: 98.1461
# val : Epoch: 188 | Accuracy: 90.4167 | best acc: 92.9167
# Train: Epoch: 189 | Loss: 0.0007 | Accuracy: 98.8004 | best acc: 98.8004
# val : Epoch: 189 | Accuracy: 90.8333 | best acc: 92.9167
# Train: Epoch: 190 | Loss: 0.0007 | Accuracy: 97.8190 | best acc: 98.8004
# val : Epoch: 190 | Accuracy: 92.5000 | best acc: 92.9167
# Train: Epoch: 191 | Loss: 0.0007 | Accuracy: 98.0371 | best acc: 98.8004
# val : Epoch: 191 | Accuracy: 88.7500 | best acc: 92.9167
# Train: Epoch: 192 | Loss: 0.0007 | Accuracy: 97.4918 | best acc: 98.8004
# val : Epoch: 192 | Accuracy: 90.0000 | best acc: 92.9167
# Train: Epoch: 193 | Loss: 0.0007 | Accuracy: 97.9280 | best acc: 98.8004
# val : Epoch: 193 | Accuracy: 91.2500 | best acc: 92.9167
# Train: Epoch: 194 | Loss: 0.0007 | Accuracy: 97.9280 | best acc: 98.8004
# val : Epoch: 194 | Accuracy: 85.8333 | best acc: 92.9167
# Train: Epoch: 195 | Loss: 0.0006 | Accuracy: 98.0371 | best 

## STD-GCN(Top1 Acc=83.3333)

In [29]:
# STD-GCN
NUM_EPOCH = 200
BATCH_SIZE = 256
acc_list2 = []
val_acc_list2 = []
best_acc = 0
val_best_acc = 0
model = STD_GCN(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list2.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list2.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.0139 | Accuracy: 4.0349 | best acc: 4.0349
# val : Epoch: 1 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 2 | Loss: 0.0137 | Accuracy: 4.0349 | best acc: 4.0349
# val : Epoch: 2 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 3 | Loss: 0.0136 | Accuracy: 5.3435 | best acc: 5.3435
# val : Epoch: 3 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 4 | Loss: 0.0133 | Accuracy: 8.1788 | best acc: 8.1788
# val : Epoch: 4 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 5 | Loss: 0.0132 | Accuracy: 7.9607 | best acc: 8.1788
# val : Epoch: 5 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 6 | Loss: 0.0131 | Accuracy: 9.9237 | best acc: 9.9237
# val : Epoch: 6 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 7 | Loss: 0.0131 | Accuracy: 10.0327 | best acc: 10.0327
# val : Epoch: 7 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 8 | Loss: 0.0130 | Accuracy: 9.7056 | best acc: 10.0327
# val : Epoch: 8 | Accuracy: 4.1667 | best a

# Train: Epoch: 64 | Loss: 0.0065 | Accuracy: 56.7067 | best acc: 56.7067
# val : Epoch: 64 | Accuracy: 28.3333 | best acc: 32.5000
# Train: Epoch: 65 | Loss: 0.0063 | Accuracy: 59.8691 | best acc: 59.8691
# val : Epoch: 65 | Accuracy: 31.2500 | best acc: 32.5000
# Train: Epoch: 66 | Loss: 0.0062 | Accuracy: 63.2497 | best acc: 63.2497
# val : Epoch: 66 | Accuracy: 30.0000 | best acc: 32.5000
# Train: Epoch: 67 | Loss: 0.0061 | Accuracy: 61.5049 | best acc: 63.2497
# val : Epoch: 67 | Accuracy: 29.5833 | best acc: 32.5000
# Train: Epoch: 68 | Loss: 0.0061 | Accuracy: 62.1592 | best acc: 63.2497
# val : Epoch: 68 | Accuracy: 35.4167 | best acc: 35.4167
# Train: Epoch: 69 | Loss: 0.0060 | Accuracy: 62.3773 | best acc: 63.2497
# val : Epoch: 69 | Accuracy: 36.6667 | best acc: 36.6667
# Train: Epoch: 70 | Loss: 0.0059 | Accuracy: 63.3588 | best acc: 63.3588
# val : Epoch: 70 | Accuracy: 36.2500 | best acc: 36.6667
# Train: Epoch: 71 | Loss: 0.0059 | Accuracy: 62.0502 | best acc: 63.3588
# 

# Train: Epoch: 126 | Loss: 0.0021 | Accuracy: 89.3130 | best acc: 89.8582
# val : Epoch: 126 | Accuracy: 59.1667 | best acc: 64.1667
# Train: Epoch: 127 | Loss: 0.0021 | Accuracy: 89.9673 | best acc: 89.9673
# val : Epoch: 127 | Accuracy: 58.7500 | best acc: 64.1667
# Train: Epoch: 128 | Loss: 0.0021 | Accuracy: 89.6401 | best acc: 89.9673
# val : Epoch: 128 | Accuracy: 60.4167 | best acc: 64.1667
# Train: Epoch: 129 | Loss: 0.0021 | Accuracy: 89.3130 | best acc: 89.9673
# val : Epoch: 129 | Accuracy: 64.5833 | best acc: 64.5833
# Train: Epoch: 130 | Loss: 0.0020 | Accuracy: 88.6587 | best acc: 89.9673
# val : Epoch: 130 | Accuracy: 52.9167 | best acc: 64.5833
# Train: Epoch: 131 | Loss: 0.0020 | Accuracy: 90.1854 | best acc: 90.1854
# val : Epoch: 131 | Accuracy: 65.8333 | best acc: 65.8333
# Train: Epoch: 132 | Loss: 0.0019 | Accuracy: 90.7306 | best acc: 90.7306
# val : Epoch: 132 | Accuracy: 56.2500 | best acc: 65.8333
# Train: Epoch: 133 | Loss: 0.0018 | Accuracy: 92.4755 | best 

# val : Epoch: 187 | Accuracy: 75.8333 | best acc: 81.6667
# Train: Epoch: 188 | Loss: 0.0008 | Accuracy: 96.6194 | best acc: 98.1461
# val : Epoch: 188 | Accuracy: 75.0000 | best acc: 81.6667
# Train: Epoch: 189 | Loss: 0.0008 | Accuracy: 96.5104 | best acc: 98.1461
# val : Epoch: 189 | Accuracy: 71.2500 | best acc: 81.6667
# Train: Epoch: 190 | Loss: 0.0008 | Accuracy: 97.1647 | best acc: 98.1461
# val : Epoch: 190 | Accuracy: 77.5000 | best acc: 81.6667
# Train: Epoch: 191 | Loss: 0.0007 | Accuracy: 98.3642 | best acc: 98.3642
# val : Epoch: 191 | Accuracy: 78.7500 | best acc: 81.6667
# Train: Epoch: 192 | Loss: 0.0008 | Accuracy: 97.3828 | best acc: 98.3642
# val : Epoch: 192 | Accuracy: 71.6667 | best acc: 81.6667
# Train: Epoch: 193 | Loss: 0.0007 | Accuracy: 98.0371 | best acc: 98.3642
# val : Epoch: 193 | Accuracy: 70.8333 | best acc: 81.6667
# Train: Epoch: 194 | Loss: 0.0007 | Accuracy: 98.0371 | best acc: 98.3642
# val : Epoch: 194 | Accuracy: 78.7500 | best acc: 81.6667
# T

## SSTD-GCN(Top1 Acc=99.1667)

In [33]:
# SSTD-GCN
NUM_EPOCH = 200
BATCH_SIZE = 128
acc_list3 = []
val_acc_list3 = []
best_acc = 0
val_best_acc = 0
model = SSTD_GCN(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.015, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list3.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list3.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.0275 | Accuracy: 4.5802 | best acc: 4.5802
# val : Epoch: 1 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 2 | Loss: 0.0264 | Accuracy: 7.8517 | best acc: 7.8517
# val : Epoch: 2 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 3 | Loss: 0.0261 | Accuracy: 8.0698 | best acc: 8.0698
# val : Epoch: 3 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 4 | Loss: 0.0257 | Accuracy: 8.2879 | best acc: 8.2879
# val : Epoch: 4 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 5 | Loss: 0.0255 | Accuracy: 8.9422 | best acc: 8.9422
# val : Epoch: 5 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 6 | Loss: 0.0250 | Accuracy: 10.9051 | best acc: 10.9051
# val : Epoch: 6 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 7 | Loss: 0.0246 | Accuracy: 13.8495 | best acc: 13.8495
# val : Epoch: 7 | Accuracy: 5.4167 | best acc: 5.4167
# Train: Epoch: 8 | Loss: 0.0241 | Accuracy: 13.3043 | best acc: 13.8495
# val : Epoch: 8 | Accuracy: 13.3333 | be

# val : Epoch: 63 | Accuracy: 82.0833 | best acc: 90.0000
# Train: Epoch: 64 | Loss: 0.0026 | Accuracy: 94.8746 | best acc: 94.8746
# val : Epoch: 64 | Accuracy: 78.3333 | best acc: 90.0000
# Train: Epoch: 65 | Loss: 0.0027 | Accuracy: 91.7121 | best acc: 94.8746
# val : Epoch: 65 | Accuracy: 85.4167 | best acc: 90.0000
# Train: Epoch: 66 | Loss: 0.0027 | Accuracy: 92.8026 | best acc: 94.8746
# val : Epoch: 66 | Accuracy: 79.5833 | best acc: 90.0000
# Train: Epoch: 67 | Loss: 0.0023 | Accuracy: 93.5660 | best acc: 94.8746
# val : Epoch: 67 | Accuracy: 80.0000 | best acc: 90.0000
# Train: Epoch: 68 | Loss: 0.0023 | Accuracy: 94.0022 | best acc: 94.8746
# val : Epoch: 68 | Accuracy: 88.7500 | best acc: 90.0000
# Train: Epoch: 69 | Loss: 0.0019 | Accuracy: 96.0742 | best acc: 96.0742
# val : Epoch: 69 | Accuracy: 87.0833 | best acc: 90.0000
# Train: Epoch: 70 | Loss: 0.0019 | Accuracy: 95.0927 | best acc: 96.0742
# val : Epoch: 70 | Accuracy: 90.8333 | best acc: 90.8333
# Train: Epoch: 71

# val : Epoch: 125 | Accuracy: 95.8333 | best acc: 97.5000
# Train: Epoch: 126 | Loss: 0.0005 | Accuracy: 98.9095 | best acc: 99.6728
# val : Epoch: 126 | Accuracy: 95.0000 | best acc: 97.5000
# Train: Epoch: 127 | Loss: 0.0006 | Accuracy: 99.0185 | best acc: 99.6728
# val : Epoch: 127 | Accuracy: 97.5000 | best acc: 97.5000
# Train: Epoch: 128 | Loss: 0.0005 | Accuracy: 99.3457 | best acc: 99.6728
# val : Epoch: 128 | Accuracy: 95.4167 | best acc: 97.5000
# Train: Epoch: 129 | Loss: 0.0004 | Accuracy: 99.1276 | best acc: 99.6728
# val : Epoch: 129 | Accuracy: 89.5833 | best acc: 97.5000
# Train: Epoch: 130 | Loss: 0.0008 | Accuracy: 99.2366 | best acc: 99.6728
# val : Epoch: 130 | Accuracy: 84.5833 | best acc: 97.5000
# Train: Epoch: 131 | Loss: 0.0008 | Accuracy: 97.8190 | best acc: 99.6728
# val : Epoch: 131 | Accuracy: 89.1667 | best acc: 97.5000
# Train: Epoch: 132 | Loss: 0.0015 | Accuracy: 98.0371 | best acc: 99.6728
# val : Epoch: 132 | Accuracy: 91.6667 | best acc: 97.5000
# T

# Train: Epoch: 187 | Loss: 0.0002 | Accuracy: 99.8909 | best acc: 99.8909
# val : Epoch: 187 | Accuracy: 98.7500 | best acc: 99.1667
# Train: Epoch: 188 | Loss: 0.0002 | Accuracy: 99.8909 | best acc: 99.8909
# val : Epoch: 188 | Accuracy: 97.9167 | best acc: 99.1667
# Train: Epoch: 189 | Loss: 0.0006 | Accuracy: 99.5638 | best acc: 99.8909
# val : Epoch: 189 | Accuracy: 99.1667 | best acc: 99.1667
# Train: Epoch: 190 | Loss: 0.0003 | Accuracy: 99.7819 | best acc: 99.8909
# val : Epoch: 190 | Accuracy: 94.5833 | best acc: 99.1667
# Train: Epoch: 191 | Loss: 0.0004 | Accuracy: 99.6728 | best acc: 99.8909
# val : Epoch: 191 | Accuracy: 95.8333 | best acc: 99.1667
# Train: Epoch: 192 | Loss: 0.0003 | Accuracy: 99.1276 | best acc: 99.8909
# val : Epoch: 192 | Accuracy: 95.0000 | best acc: 99.1667
# Train: Epoch: 193 | Loss: 0.0004 | Accuracy: 99.4547 | best acc: 99.8909
# val : Epoch: 193 | Accuracy: 96.6667 | best acc: 99.1667
# Train: Epoch: 194 | Loss: 0.0004 | Accuracy: 99.1276 | best 

## STGL-GCN(Top1 Acc=94.1667)

In [20]:
# STGL-GCN
NUM_EPOCH = 200
BATCH_SIZE = 128
acc_list4 = []
val_acc_list4 = []
best_acc = 0
val_best_acc = 0
model = STGL_GCN(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.015)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list4.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list4.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.0268 | Accuracy: 6.6521 | best acc: 6.6521
# val : Epoch: 1 | Accuracy: 5.8333 | best acc: 5.8333
# Train: Epoch: 2 | Loss: 0.0260 | Accuracy: 9.7056 | best acc: 9.7056
# val : Epoch: 2 | Accuracy: 5.4167 | best acc: 5.8333
# Train: Epoch: 3 | Loss: 0.0253 | Accuracy: 17.2301 | best acc: 17.2301
# val : Epoch: 3 | Accuracy: 10.8333 | best acc: 10.8333
# Train: Epoch: 4 | Loss: 0.0244 | Accuracy: 22.5736 | best acc: 22.5736
# val : Epoch: 4 | Accuracy: 17.5000 | best acc: 17.5000
# Train: Epoch: 5 | Loss: 0.0239 | Accuracy: 25.0818 | best acc: 25.0818
# val : Epoch: 5 | Accuracy: 17.5000 | best acc: 17.5000
# Train: Epoch: 6 | Loss: 0.0231 | Accuracy: 27.1538 | best acc: 27.1538
# val : Epoch: 6 | Accuracy: 14.5833 | best acc: 17.5000
# Train: Epoch: 7 | Loss: 0.0223 | Accuracy: 29.9891 | best acc: 29.9891
# val : Epoch: 7 | Accuracy: 19.1667 | best acc: 19.1667
# Train: Epoch: 8 | Loss: 0.0217 | Accuracy: 29.9891 | best acc: 29.9891
# val : Epoch: 8 | Accura

# Train: Epoch: 64 | Loss: 0.0073 | Accuracy: 76.0087 | best acc: 77.3173
# val : Epoch: 64 | Accuracy: 61.2500 | best acc: 61.2500
# Train: Epoch: 65 | Loss: 0.0073 | Accuracy: 78.1897 | best acc: 78.1897
# val : Epoch: 65 | Accuracy: 42.9167 | best acc: 61.2500
# Train: Epoch: 66 | Loss: 0.0075 | Accuracy: 77.8626 | best acc: 78.1897
# val : Epoch: 66 | Accuracy: 60.4167 | best acc: 61.2500
# Train: Epoch: 67 | Loss: 0.0063 | Accuracy: 82.4427 | best acc: 82.4427
# val : Epoch: 67 | Accuracy: 50.4167 | best acc: 61.2500
# Train: Epoch: 68 | Loss: 0.0070 | Accuracy: 79.2803 | best acc: 82.4427
# val : Epoch: 68 | Accuracy: 47.0833 | best acc: 61.2500
# Train: Epoch: 69 | Loss: 0.0063 | Accuracy: 82.4427 | best acc: 82.4427
# val : Epoch: 69 | Accuracy: 49.5833 | best acc: 61.2500
# Train: Epoch: 70 | Loss: 0.0062 | Accuracy: 81.7884 | best acc: 82.4427
# val : Epoch: 70 | Accuracy: 53.3333 | best acc: 61.2500
# Train: Epoch: 71 | Loss: 0.0071 | Accuracy: 82.6609 | best acc: 82.6609
# 

# Train: Epoch: 126 | Loss: 0.0023 | Accuracy: 95.3108 | best acc: 95.6379
# val : Epoch: 126 | Accuracy: 71.2500 | best acc: 71.2500
# Train: Epoch: 127 | Loss: 0.0022 | Accuracy: 94.8746 | best acc: 95.6379
# val : Epoch: 127 | Accuracy: 63.7500 | best acc: 71.2500
# Train: Epoch: 128 | Loss: 0.0023 | Accuracy: 95.0927 | best acc: 95.6379
# val : Epoch: 128 | Accuracy: 53.7500 | best acc: 71.2500
# Train: Epoch: 129 | Loss: 0.0027 | Accuracy: 93.4569 | best acc: 95.6379
# val : Epoch: 129 | Accuracy: 51.6667 | best acc: 71.2500
# Train: Epoch: 130 | Loss: 0.0023 | Accuracy: 95.6379 | best acc: 95.6379
# val : Epoch: 130 | Accuracy: 67.5000 | best acc: 71.2500
# Train: Epoch: 131 | Loss: 0.0020 | Accuracy: 96.4013 | best acc: 96.4013
# val : Epoch: 131 | Accuracy: 57.0833 | best acc: 71.2500
# Train: Epoch: 132 | Loss: 0.0018 | Accuracy: 96.5104 | best acc: 96.5104
# val : Epoch: 132 | Accuracy: 59.5833 | best acc: 71.2500
# Train: Epoch: 133 | Loss: 0.0020 | Accuracy: 96.1832 | best 

# Train: Epoch: 188 | Loss: 0.0012 | Accuracy: 98.1461 | best acc: 99.5638
# val : Epoch: 188 | Accuracy: 62.0833 | best acc: 74.5833
# Train: Epoch: 189 | Loss: 0.0010 | Accuracy: 98.5823 | best acc: 99.5638
# val : Epoch: 189 | Accuracy: 70.0000 | best acc: 74.5833
# Train: Epoch: 190 | Loss: 0.0012 | Accuracy: 99.4547 | best acc: 99.5638
# val : Epoch: 190 | Accuracy: 52.0833 | best acc: 74.5833
# Train: Epoch: 191 | Loss: 0.0021 | Accuracy: 94.9836 | best acc: 99.5638
# val : Epoch: 191 | Accuracy: 59.1667 | best acc: 74.5833
# Train: Epoch: 192 | Loss: 0.0012 | Accuracy: 97.6009 | best acc: 99.5638
# val : Epoch: 192 | Accuracy: 71.2500 | best acc: 74.5833
# Train: Epoch: 193 | Loss: 0.0007 | Accuracy: 99.4547 | best acc: 99.5638
# val : Epoch: 193 | Accuracy: 72.5000 | best acc: 74.5833
# Train: Epoch: 194 | Loss: 0.0008 | Accuracy: 99.3457 | best acc: 99.5638
# val : Epoch: 194 | Accuracy: 67.5000 | best acc: 74.5833
# Train: Epoch: 195 | Loss: 0.0007 | Accuracy: 99.4547 | best 

## ST-GCN++(Top1 Acc=    )

In [28]:
# STGL-GCN
NUM_EPOCH = 300
BATCH_SIZE = 64
acc_list5 = []
val_acc_list5 = []
best_acc = 0
val_best_acc = 0
model = ST_GCNPP(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list5.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list5.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.0521 | Accuracy: 7.0883 | best acc: 7.0883
# val : Epoch: 1 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 2 | Loss: 0.0513 | Accuracy: 10.7961 | best acc: 10.7961
# val : Epoch: 2 | Accuracy: 4.5833 | best acc: 4.5833
# Train: Epoch: 3 | Loss: 0.0507 | Accuracy: 13.8495 | best acc: 13.8495
# val : Epoch: 3 | Accuracy: 15.8333 | best acc: 15.8333
# Train: Epoch: 4 | Loss: 0.0500 | Accuracy: 18.6478 | best acc: 18.6478
# val : Epoch: 4 | Accuracy: 20.8333 | best acc: 20.8333
# Train: Epoch: 5 | Loss: 0.0495 | Accuracy: 20.7197 | best acc: 20.7197
# val : Epoch: 5 | Accuracy: 23.7500 | best acc: 23.7500
# Train: Epoch: 6 | Loss: 0.0489 | Accuracy: 22.7917 | best acc: 22.7917
# val : Epoch: 6 | Accuracy: 26.2500 | best acc: 26.2500
# Train: Epoch: 7 | Loss: 0.0484 | Accuracy: 24.4275 | best acc: 24.4275
# val : Epoch: 7 | Accuracy: 27.9167 | best acc: 27.9167
# Train: Epoch: 8 | Loss: 0.0479 | Accuracy: 25.9542 | best acc: 25.9542
# val : Epoch: 8 | Accu

# val : Epoch: 63 | Accuracy: 48.7500 | best acc: 49.5833
# Train: Epoch: 64 | Loss: 0.0295 | Accuracy: 51.5812 | best acc: 52.5627
# val : Epoch: 64 | Accuracy: 50.0000 | best acc: 50.0000
# Train: Epoch: 65 | Loss: 0.0293 | Accuracy: 51.7993 | best acc: 52.5627
# val : Epoch: 65 | Accuracy: 52.5000 | best acc: 52.5000
# Train: Epoch: 66 | Loss: 0.0291 | Accuracy: 54.9618 | best acc: 54.9618
# val : Epoch: 66 | Accuracy: 51.2500 | best acc: 52.5000
# Train: Epoch: 67 | Loss: 0.0285 | Accuracy: 54.8528 | best acc: 54.9618
# val : Epoch: 67 | Accuracy: 52.5000 | best acc: 52.5000
# Train: Epoch: 68 | Loss: 0.0283 | Accuracy: 55.0709 | best acc: 55.0709
# val : Epoch: 68 | Accuracy: 50.0000 | best acc: 52.5000
# Train: Epoch: 69 | Loss: 0.0282 | Accuracy: 53.9804 | best acc: 55.0709
# val : Epoch: 69 | Accuracy: 52.5000 | best acc: 52.5000
# Train: Epoch: 70 | Loss: 0.0279 | Accuracy: 56.4885 | best acc: 56.4885
# val : Epoch: 70 | Accuracy: 51.2500 | best acc: 52.5000
# Train: Epoch: 71

# val : Epoch: 125 | Accuracy: 75.8333 | best acc: 76.6667
# Train: Epoch: 126 | Loss: 0.0175 | Accuracy: 77.2083 | best acc: 79.2803
# val : Epoch: 126 | Accuracy: 56.2500 | best acc: 76.6667
# Train: Epoch: 127 | Loss: 0.0173 | Accuracy: 78.8441 | best acc: 79.2803
# val : Epoch: 127 | Accuracy: 60.0000 | best acc: 76.6667
# Train: Epoch: 128 | Loss: 0.0174 | Accuracy: 76.5540 | best acc: 79.2803
# val : Epoch: 128 | Accuracy: 76.2500 | best acc: 76.6667
# Train: Epoch: 129 | Loss: 0.0169 | Accuracy: 78.9531 | best acc: 79.2803
# val : Epoch: 129 | Accuracy: 70.0000 | best acc: 76.6667
# Train: Epoch: 130 | Loss: 0.0169 | Accuracy: 79.2803 | best acc: 79.2803
# val : Epoch: 130 | Accuracy: 68.3333 | best acc: 76.6667
# Train: Epoch: 131 | Loss: 0.0171 | Accuracy: 77.7535 | best acc: 79.2803
# val : Epoch: 131 | Accuracy: 66.6667 | best acc: 76.6667
# Train: Epoch: 132 | Loss: 0.0168 | Accuracy: 77.9716 | best acc: 79.2803
# val : Epoch: 132 | Accuracy: 78.3333 | best acc: 78.3333
# T

# Train: Epoch: 187 | Loss: 0.0114 | Accuracy: 88.2225 | best acc: 88.3315
# val : Epoch: 187 | Accuracy: 68.3333 | best acc: 84.5833
# Train: Epoch: 188 | Loss: 0.0116 | Accuracy: 86.0414 | best acc: 88.3315
# val : Epoch: 188 | Accuracy: 72.9167 | best acc: 84.5833
# Train: Epoch: 189 | Loss: 0.0113 | Accuracy: 86.4776 | best acc: 88.3315
# val : Epoch: 189 | Accuracy: 83.3333 | best acc: 84.5833
# Train: Epoch: 190 | Loss: 0.0112 | Accuracy: 87.2410 | best acc: 88.3315
# val : Epoch: 190 | Accuracy: 80.8333 | best acc: 84.5833
# Train: Epoch: 191 | Loss: 0.0110 | Accuracy: 87.2410 | best acc: 88.3315
# val : Epoch: 191 | Accuracy: 79.5833 | best acc: 84.5833
# Train: Epoch: 192 | Loss: 0.0109 | Accuracy: 87.6772 | best acc: 88.3315
# val : Epoch: 192 | Accuracy: 79.1667 | best acc: 84.5833
# Train: Epoch: 193 | Loss: 0.0110 | Accuracy: 88.5496 | best acc: 88.5496
# val : Epoch: 193 | Accuracy: 80.0000 | best acc: 84.5833
# Train: Epoch: 194 | Loss: 0.0106 | Accuracy: 87.4591 | best 

# val : Epoch: 248 | Accuracy: 88.7500 | best acc: 88.7500
# Train: Epoch: 249 | Loss: 0.0073 | Accuracy: 93.5660 | best acc: 93.5660
# val : Epoch: 249 | Accuracy: 87.5000 | best acc: 88.7500
# Train: Epoch: 250 | Loss: 0.0079 | Accuracy: 91.2759 | best acc: 93.5660
# val : Epoch: 250 | Accuracy: 85.4167 | best acc: 88.7500
# Train: Epoch: 251 | Loss: 0.0075 | Accuracy: 92.1483 | best acc: 93.5660
# val : Epoch: 251 | Accuracy: 87.5000 | best acc: 88.7500
# Train: Epoch: 252 | Loss: 0.0077 | Accuracy: 91.4940 | best acc: 93.5660
# val : Epoch: 252 | Accuracy: 86.6667 | best acc: 88.7500
# Train: Epoch: 253 | Loss: 0.0076 | Accuracy: 92.3664 | best acc: 93.5660
# val : Epoch: 253 | Accuracy: 87.5000 | best acc: 88.7500
# Train: Epoch: 254 | Loss: 0.0080 | Accuracy: 90.5125 | best acc: 93.5660
# val : Epoch: 254 | Accuracy: 72.9167 | best acc: 88.7500
# Train: Epoch: 255 | Loss: 0.0078 | Accuracy: 90.9487 | best acc: 93.5660
# val : Epoch: 255 | Accuracy: 87.0833 | best acc: 88.7500
# T

In [30]:
# STGL-GCN
NUM_EPOCH = 300
BATCH_SIZE = 64
acc_list5 = []
val_acc_list5 = []
best_acc = 0
val_best_acc = 0
model = ST_GCNPP(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list5.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list5.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.0516 | Accuracy: 9.2694 | best acc: 9.2694
# val : Epoch: 1 | Accuracy: 2.5000 | best acc: 2.5000
# Train: Epoch: 2 | Loss: 0.0506 | Accuracy: 11.4504 | best acc: 11.4504
# val : Epoch: 2 | Accuracy: 9.5833 | best acc: 9.5833
# Train: Epoch: 3 | Loss: 0.0498 | Accuracy: 13.9586 | best acc: 13.9586
# val : Epoch: 3 | Accuracy: 18.7500 | best acc: 18.7500
# Train: Epoch: 4 | Loss: 0.0490 | Accuracy: 17.9935 | best acc: 17.9935
# val : Epoch: 4 | Accuracy: 20.4167 | best acc: 20.4167
# Train: Epoch: 5 | Loss: 0.0484 | Accuracy: 19.1930 | best acc: 19.1930
# val : Epoch: 5 | Accuracy: 20.0000 | best acc: 20.4167
# Train: Epoch: 6 | Loss: 0.0479 | Accuracy: 22.4646 | best acc: 22.4646
# val : Epoch: 6 | Accuracy: 24.1667 | best acc: 24.1667
# Train: Epoch: 7 | Loss: 0.0472 | Accuracy: 23.9913 | best acc: 23.9913
# val : Epoch: 7 | Accuracy: 25.4167 | best acc: 25.4167
# Train: Epoch: 8 | Loss: 0.0467 | Accuracy: 24.7546 | best acc: 24.7546
# val : Epoch: 8 | Accu

# val : Epoch: 63 | Accuracy: 47.0833 | best acc: 51.2500
# Train: Epoch: 64 | Loss: 0.0298 | Accuracy: 52.7808 | best acc: 52.7808
# val : Epoch: 64 | Accuracy: 50.4167 | best acc: 51.2500
# Train: Epoch: 65 | Loss: 0.0296 | Accuracy: 52.9989 | best acc: 52.9989
# val : Epoch: 65 | Accuracy: 52.9167 | best acc: 52.9167
# Train: Epoch: 66 | Loss: 0.0293 | Accuracy: 54.0894 | best acc: 54.0894
# val : Epoch: 66 | Accuracy: 50.8333 | best acc: 52.9167
# Train: Epoch: 67 | Loss: 0.0292 | Accuracy: 54.5256 | best acc: 54.5256
# val : Epoch: 67 | Accuracy: 50.8333 | best acc: 52.9167
# Train: Epoch: 68 | Loss: 0.0292 | Accuracy: 55.0709 | best acc: 55.0709
# val : Epoch: 68 | Accuracy: 52.5000 | best acc: 52.9167
# Train: Epoch: 69 | Loss: 0.0291 | Accuracy: 55.0709 | best acc: 55.0709
# val : Epoch: 69 | Accuracy: 52.0833 | best acc: 52.9167
# Train: Epoch: 70 | Loss: 0.0282 | Accuracy: 57.9062 | best acc: 57.9062
# val : Epoch: 70 | Accuracy: 51.2500 | best acc: 52.9167
# Train: Epoch: 71

# val : Epoch: 125 | Accuracy: 78.3333 | best acc: 78.3333
# Train: Epoch: 126 | Loss: 0.0169 | Accuracy: 79.1712 | best acc: 80.1527
# val : Epoch: 126 | Accuracy: 75.4167 | best acc: 78.3333
# Train: Epoch: 127 | Loss: 0.0167 | Accuracy: 79.2803 | best acc: 80.1527
# val : Epoch: 127 | Accuracy: 69.5833 | best acc: 78.3333
# Train: Epoch: 128 | Loss: 0.0169 | Accuracy: 79.3893 | best acc: 80.1527
# val : Epoch: 128 | Accuracy: 67.0833 | best acc: 78.3333
# Train: Epoch: 129 | Loss: 0.0164 | Accuracy: 80.1527 | best acc: 80.1527
# val : Epoch: 129 | Accuracy: 74.1667 | best acc: 78.3333
# Train: Epoch: 130 | Loss: 0.0162 | Accuracy: 80.6979 | best acc: 80.6979
# val : Epoch: 130 | Accuracy: 73.7500 | best acc: 78.3333
# Train: Epoch: 131 | Loss: 0.0164 | Accuracy: 78.4079 | best acc: 80.6979
# val : Epoch: 131 | Accuracy: 68.7500 | best acc: 78.3333
# Train: Epoch: 132 | Loss: 0.0160 | Accuracy: 78.0807 | best acc: 80.6979
# val : Epoch: 132 | Accuracy: 75.4167 | best acc: 78.3333
# T

# Train: Epoch: 187 | Loss: 0.0113 | Accuracy: 87.1320 | best acc: 88.8768
# val : Epoch: 187 | Accuracy: 78.7500 | best acc: 87.0833
# Train: Epoch: 188 | Loss: 0.0106 | Accuracy: 88.3315 | best acc: 88.8768
# val : Epoch: 188 | Accuracy: 77.5000 | best acc: 87.0833
# Train: Epoch: 189 | Loss: 0.0105 | Accuracy: 88.8768 | best acc: 88.8768
# val : Epoch: 189 | Accuracy: 87.5000 | best acc: 87.5000
# Train: Epoch: 190 | Loss: 0.0107 | Accuracy: 87.5682 | best acc: 88.8768
# val : Epoch: 190 | Accuracy: 78.7500 | best acc: 87.5000
# Train: Epoch: 191 | Loss: 0.0109 | Accuracy: 89.6401 | best acc: 89.6401
# val : Epoch: 191 | Accuracy: 75.8333 | best acc: 87.5000
# Train: Epoch: 192 | Loss: 0.0111 | Accuracy: 86.8048 | best acc: 89.6401
# val : Epoch: 192 | Accuracy: 84.1667 | best acc: 87.5000
# Train: Epoch: 193 | Loss: 0.0101 | Accuracy: 89.7492 | best acc: 89.7492
# val : Epoch: 193 | Accuracy: 75.8333 | best acc: 87.5000
# Train: Epoch: 194 | Loss: 0.0107 | Accuracy: 87.6772 | best 

# val : Epoch: 248 | Accuracy: 82.0833 | best acc: 90.0000
# Train: Epoch: 249 | Loss: 0.0073 | Accuracy: 92.6936 | best acc: 93.8931
# val : Epoch: 249 | Accuracy: 86.2500 | best acc: 90.0000
# Train: Epoch: 250 | Loss: 0.0071 | Accuracy: 93.4569 | best acc: 93.8931
# val : Epoch: 250 | Accuracy: 88.3333 | best acc: 90.0000
# Train: Epoch: 251 | Loss: 0.0077 | Accuracy: 92.4755 | best acc: 93.8931
# val : Epoch: 251 | Accuracy: 80.4167 | best acc: 90.0000
# Train: Epoch: 252 | Loss: 0.0073 | Accuracy: 92.6936 | best acc: 93.8931
# val : Epoch: 252 | Accuracy: 90.4167 | best acc: 90.4167
# Train: Epoch: 253 | Loss: 0.0069 | Accuracy: 93.5660 | best acc: 93.8931
# val : Epoch: 253 | Accuracy: 89.1667 | best acc: 90.4167
# Train: Epoch: 254 | Loss: 0.0071 | Accuracy: 91.4940 | best acc: 93.8931
# val : Epoch: 254 | Accuracy: 92.5000 | best acc: 92.5000
# Train: Epoch: 255 | Loss: 0.0071 | Accuracy: 93.0207 | best acc: 93.8931
# val : Epoch: 255 | Accuracy: 84.1667 | best acc: 92.5000
# T

## SSTD-GCN++(Top1 Acc=97.5   )

In [25]:
NUM_EPOCH = 300
BATCH_SIZE = 32
acc_list6 = []
val_acc_list6 = []
best_acc = 0
val_best_acc = 0
model = SSTD_GCNPP(num_classes=24,
                  in_channels=3,
                  t_kernel_size=9,
                  hop_size=1).cuda()


optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

data_loader = dict()
data_loader['train'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/train_inputs.npy', label_path='AI/Taichi data/train_labels.npy'), batch_size=BATCH_SIZE, shuffle=True)
data_loader['test'] = torch.utils.data.DataLoader(dataset=Feeder(data_path='AI/Taichi data/val_inputs.npy', label_path='AI/Taichi data/val_labels.npy'), batch_size=BATCH_SIZE, shuffle=False)


for epoch in range(1, NUM_EPOCH+1):
  model.train()
  correct = 0
  sum_loss = 0
  val_correct = 0
  for batch_idx, (data, label) in enumerate(data_loader['train']):
    data = data.cuda()
    label = label[:,1].long().cuda()
    output = model(data)

    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    sum_loss += loss.item()
    _, predict = torch.max(output.data, 1)
    correct += (predict == label).sum().item()
  acc = (100. * correct / len(data_loader['train'].dataset))
  acc_list6.append(acc)
  if acc > best_acc:
    best_acc = acc
  print('# Train: Epoch: {} | Loss: {:.4f} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, sum_loss/len(data_loader['train'].dataset), acc, best_acc))
  with torch.no_grad():
    model.eval()
    for batch_idx, (data, label) in enumerate(data_loader['test']):
      data = data.cuda()
      label = label[:,1].long().cuda()
      output = model(data)

      _, predict = torch.max(output.data, 1)
      val_correct += (predict == label).sum().item()
  val_acc = (100. * val_correct / len(data_loader['test'].dataset))
  val_acc_list6.append(val_acc)
  if val_acc > val_best_acc:
    val_best_acc = val_acc
  print('# val : Epoch: {} | Accuracy: {:.4f} | best acc: {:.4f}'.format(epoch, val_acc, val_best_acc))

# Train: Epoch: 1 | Loss: 0.1003 | Accuracy: 4.2530 | best acc: 4.2530
# val : Epoch: 1 | Accuracy: 4.1667 | best acc: 4.1667
# Train: Epoch: 2 | Loss: 0.0975 | Accuracy: 6.7612 | best acc: 6.7612
# val : Epoch: 2 | Accuracy: 9.1667 | best acc: 9.1667
# Train: Epoch: 3 | Loss: 0.0949 | Accuracy: 13.7405 | best acc: 13.7405
# val : Epoch: 3 | Accuracy: 17.5000 | best acc: 17.5000
# Train: Epoch: 4 | Loss: 0.0925 | Accuracy: 21.7012 | best acc: 21.7012
# val : Epoch: 4 | Accuracy: 19.5833 | best acc: 19.5833
# Train: Epoch: 5 | Loss: 0.0906 | Accuracy: 26.7176 | best acc: 26.7176
# val : Epoch: 5 | Accuracy: 27.0833 | best acc: 27.0833
# Train: Epoch: 6 | Loss: 0.0886 | Accuracy: 28.7895 | best acc: 28.7895
# val : Epoch: 6 | Accuracy: 30.0000 | best acc: 30.0000
# Train: Epoch: 7 | Loss: 0.0868 | Accuracy: 29.0076 | best acc: 29.0076
# val : Epoch: 7 | Accuracy: 28.7500 | best acc: 30.0000
# Train: Epoch: 8 | Loss: 0.0850 | Accuracy: 30.8615 | best acc: 30.8615
# val : Epoch: 8 | Accura

# val : Epoch: 63 | Accuracy: 69.5833 | best acc: 75.4167
# Train: Epoch: 64 | Loss: 0.0325 | Accuracy: 75.5725 | best acc: 75.5725
# val : Epoch: 64 | Accuracy: 71.2500 | best acc: 75.4167
# Train: Epoch: 65 | Loss: 0.0316 | Accuracy: 74.7001 | best acc: 75.5725
# val : Epoch: 65 | Accuracy: 80.4167 | best acc: 80.4167
# Train: Epoch: 66 | Loss: 0.0307 | Accuracy: 76.8811 | best acc: 76.8811
# val : Epoch: 66 | Accuracy: 75.4167 | best acc: 80.4167
# Train: Epoch: 67 | Loss: 0.0304 | Accuracy: 77.8626 | best acc: 77.8626
# val : Epoch: 67 | Accuracy: 75.8333 | best acc: 80.4167
# Train: Epoch: 68 | Loss: 0.0299 | Accuracy: 78.9531 | best acc: 78.9531
# val : Epoch: 68 | Accuracy: 68.3333 | best acc: 80.4167
# Train: Epoch: 69 | Loss: 0.0295 | Accuracy: 78.4079 | best acc: 78.9531
# val : Epoch: 69 | Accuracy: 70.8333 | best acc: 80.4167
# Train: Epoch: 70 | Loss: 0.0288 | Accuracy: 78.1897 | best acc: 78.9531
# val : Epoch: 70 | Accuracy: 79.1667 | best acc: 80.4167
# Train: Epoch: 71

# val : Epoch: 125 | Accuracy: 82.0833 | best acc: 88.7500
# Train: Epoch: 126 | Loss: 0.0155 | Accuracy: 90.9487 | best acc: 90.9487
# val : Epoch: 126 | Accuracy: 69.1667 | best acc: 88.7500
# Train: Epoch: 127 | Loss: 0.0167 | Accuracy: 88.2225 | best acc: 90.9487
# val : Epoch: 127 | Accuracy: 82.0833 | best acc: 88.7500
# Train: Epoch: 128 | Loss: 0.0164 | Accuracy: 89.5311 | best acc: 90.9487
# val : Epoch: 128 | Accuracy: 86.6667 | best acc: 88.7500
# Train: Epoch: 129 | Loss: 0.0151 | Accuracy: 90.9487 | best acc: 90.9487
# val : Epoch: 129 | Accuracy: 87.5000 | best acc: 88.7500
# Train: Epoch: 130 | Loss: 0.0148 | Accuracy: 92.1483 | best acc: 92.1483
# val : Epoch: 130 | Accuracy: 79.5833 | best acc: 88.7500
# Train: Epoch: 131 | Loss: 0.0153 | Accuracy: 90.1854 | best acc: 92.1483
# val : Epoch: 131 | Accuracy: 82.0833 | best acc: 88.7500
# Train: Epoch: 132 | Loss: 0.0151 | Accuracy: 90.1854 | best acc: 92.1483
# val : Epoch: 132 | Accuracy: 88.7500 | best acc: 88.7500
# T

# Train: Epoch: 187 | Loss: 0.0088 | Accuracy: 95.2017 | best acc: 95.4198
# val : Epoch: 187 | Accuracy: 89.5833 | best acc: 92.0833
# Train: Epoch: 188 | Loss: 0.0083 | Accuracy: 94.9836 | best acc: 95.4198
# val : Epoch: 188 | Accuracy: 90.8333 | best acc: 92.0833
# Train: Epoch: 189 | Loss: 0.0084 | Accuracy: 95.0927 | best acc: 95.4198
# val : Epoch: 189 | Accuracy: 85.4167 | best acc: 92.0833
# Train: Epoch: 190 | Loss: 0.0078 | Accuracy: 95.6379 | best acc: 95.6379
# val : Epoch: 190 | Accuracy: 92.5000 | best acc: 92.5000
# Train: Epoch: 191 | Loss: 0.0082 | Accuracy: 94.8746 | best acc: 95.6379
# val : Epoch: 191 | Accuracy: 71.6667 | best acc: 92.5000
# Train: Epoch: 192 | Loss: 0.0081 | Accuracy: 95.4198 | best acc: 95.6379
# val : Epoch: 192 | Accuracy: 86.6667 | best acc: 92.5000
# Train: Epoch: 193 | Loss: 0.0089 | Accuracy: 93.7841 | best acc: 95.6379
# val : Epoch: 193 | Accuracy: 93.3333 | best acc: 93.3333
# Train: Epoch: 194 | Loss: 0.0075 | Accuracy: 96.4013 | best 

# val : Epoch: 248 | Accuracy: 79.1667 | best acc: 95.4167
# Train: Epoch: 249 | Loss: 0.0052 | Accuracy: 97.7099 | best acc: 98.5823
# val : Epoch: 249 | Accuracy: 90.8333 | best acc: 95.4167
# Train: Epoch: 250 | Loss: 0.0051 | Accuracy: 97.6009 | best acc: 98.5823
# val : Epoch: 250 | Accuracy: 94.1667 | best acc: 95.4167
# Train: Epoch: 251 | Loss: 0.0052 | Accuracy: 96.9466 | best acc: 98.5823
# val : Epoch: 251 | Accuracy: 90.0000 | best acc: 95.4167
# Train: Epoch: 252 | Loss: 0.0048 | Accuracy: 97.6009 | best acc: 98.5823
# val : Epoch: 252 | Accuracy: 93.7500 | best acc: 95.4167
# Train: Epoch: 253 | Loss: 0.0048 | Accuracy: 97.8190 | best acc: 98.5823
# val : Epoch: 253 | Accuracy: 92.0833 | best acc: 95.4167
# Train: Epoch: 254 | Loss: 0.0045 | Accuracy: 97.7099 | best acc: 98.5823
# val : Epoch: 254 | Accuracy: 84.1667 | best acc: 95.4167
# Train: Epoch: 255 | Loss: 0.0040 | Accuracy: 98.4733 | best acc: 98.5823
# val : Epoch: 255 | Accuracy: 93.3333 | best acc: 95.4167
# T

# valid set iteration curve

In [ ]:
x = range(200)
plt.plot(x,val_acc_list,label="ST-GCN")
plt.plot(x,val_acc_list1,label="SST-GCN")
plt.plot(x,val_acc_list2,label="STD-GCN")
plt.plot(x,val_acc_list3,label="SSTD-GCN")
plt.legend()
plt.title("Taichi Accuracy")
plt.xlabel("epoch")
plt.ylabel("Acc")

In [ ]:
"""
Supplementary Experiment 1: STGL-GCN
"""
x = range(200)
plt.plot(x,val_acc_list4,label="STGL-GCN")
plt.legend()
plt.title("NTU-RGB-D Accuracy")
plt.xlabel("epoch")
plt.ylabel("Acc")

In [ ]:
"""
Supplementary Experiment 2 and 3: ST-GCN++ and SSTD-GCN++
"""
x = range(200)
plt.plot(x,val_acc_list5,label="ST-GCN++")
plt.plot(x,val_acc_list6,label="SSTD-GCN++")
plt.legend()
plt.title("NTU-RGB-D Accuracy")
plt.xlabel("epoch")
plt.ylabel("Acc")